This file fetches word content from the wiktionary articles

It can be used to get the content of a single word (1) or from a file with multiple words

In [11]:
from python_files.getwrdlist import *
from python_files.get_html import *
from python_files.get_table_fun import *

from bs4 import BeautifulSoup
from IPython.display import clear_output
import requests
import pyperclip
import numpy as np
import requests
import re
import datetime
from googletrans import Translator

def getHTMLCode(url):
    try:
        req = requests.get(url, timeout=30)
        req.encoding = 'utf-8' 
        print("Processing fetch ...")
        return BeautifulSoup(req.text, 'html.parser')
    except:
        print("Retrying ...")
        return getHTMLCode(url)

In [12]:
## GET BIGGEST ID
def getMaxID():
    maxID = 0
    with open(WORDLIST_PATH,"r",encoding="UTF-8") as f:
        for line in f:
            if line.startswith("<ID>"):
                line = line[line.index("<ID>")+4:line.index("</ID>")]
                if int(line) > maxID:
                    maxID = int(line)

    return maxID

print(getMaxID())

43148


In [13]:
def getWordContent(name, elang = "sh", defn = "?"):
    ID = getMaxID() + 1

    soup = getHTMLCode("https://" + elang + ".wiktionary.org/wiki/" + name)
    #soup = soup.find("span", {"id" : "Serbo-Croatian"})
    tables = ""
    if "Pridjev" in soup.text or "Adjective" in soup.text:
        try:
            tables = GET_ADJECTIVE(soup)
        except:
            tables = ""
            
        type_ = "Adjective"
    elif "Glagol" in soup.text or "Verb" in soup.text:
        try:
            tables = GET_VERB(soup)
        except:
            tables = ""
        aspect = ""
        if len(soup.findAll("span", {"class" : "aspect impf"})) > 0 and len(soup.findAll("span", {"class" : "aspect pf"})) > 0:
            aspect = "imperfective or perfective"
        elif len(soup.findAll("span", {"class" : "aspect impf"})) > 0:
            aspect = "imperfective"
        elif len(soup.findAll("span", {"class" : "aspect pf"})) > 0:
            aspect = "perfective"
        type_ = "Verb (" + aspect + ")"
    elif "Imenica" in soup.text or "imenice" in soup.text or "Noun" in soup.text:
        try:
            tables = GET_NOUN(soup)
        except:
            tables = ""
        gender = soup.find("span", {"class" : "gender"}).text.strip()
        type_ = "Noun (" + gender + ")"
    elif "Prilog" in soup.text or "Adverb" in soup.text:
        tables = ""
        type_ = "Adverb"
    else:
        type_ = ""
        tables = ""

    ctr = word_scheme(tables,name,type_,ID,defn)
    
    return ctr

In [14]:
def getGlosbeDef(word):
    soup = getHTMLCode("https://glosbe.com/hr/en/" + word)
    if "We are sorry, but we have no translations for" in soup.text.strip():
        return ""
    
    soup = soup.findAll("strong", {"class" : "phr"})
    
    en = ""
    i = 0
    for s in soup:
        i += 1
        en += s.text.strip() + ", "
        
        if i == 4:
            break
    
    return en[0:-2]

def getGoogleTranslate(word):
    translator = Translator()
    translation = translator.translate(word, src='hr')
    return translation.text
    
print(getGoogleTranslate("Argentinac"))
print(getGlosbeDef("Argentinac"))

Argentinian
Processing fetch ...
Argentinian


<hr>

<h2>Get single word</h2>

In [16]:
ctr = getWordContent("deminirati", defn = ["demine (clear of mines, make the mines no longer explode or disappear)"])
pyperclip.copy(ctr)
print(ctr)

Processing fetch ...
<word>
<ID>43150</ID>
<name>deminirati</name>
<type>Verb (perfective)</type>
<webpage>,,http://sh.wiktionary.org/wiki/deminirati</webpage>
<originaldef></originaldef>
<synonyms></synonyms>
<relatedwords></relatedwords>
<english>
<en>demine (clear of mines, make the mines no longer explode or disappear)</en>
</english>
<examples>
</examples>
<tables>
<tb>Past participle</tb>
<content>null,Masculine singular,Feminine singular,Neuter singular,Masculine plural,Feminine plural,Neuter plural,,Active past participle,deminirao,deminirala,deminiralo,deminirali,deminirale,deminirala,,Passive past participle,deminiran,deminirana,deminirano,deminirani,deminirane,deminirana</content>
<tb>Conjugation</tb>
<content>Verbal forms,ja,ti,on / ona / ono,mi,vi,oni / one / ona,,Present,deminiram,deminiraš,deminira,deminiramo,deminirate,deminiraju,,Future I,deminirat ću / deminiraću,deminirat ćeš / deminiraćeš,deminirat će / deminiraće,deminirat ćemo / deminiraćemo,deminirat ćete / demin

<h2>Get words from file</h2>

In [ ]:
FILE_INPUT = "input_words.txt"
FILE_OUTPUT = "output_words.xml"

output = open(FILE_OUTPUT,"w+",encoding = "UTF-8")

i = 0 #word counter
s = 0 #successfuly added words counter
error_words = []

with open(FILE_INPUT,"r",encoding="UTF-8") as f:
    for line in f:
        i += 1
        line = line.replace("\n","")
        
        defn = getGoogleTranslate(line) 
        ctr  = getWordContent(word)
        
        if "<type></type>" in ctr or defn == line:
            continue
            
        s += 1
        output.write(ctr)
        
        clear_output(wait = True)
        print("total: " + str(i) + "    success: " + str(s))
        
    f.close()
    output.close()
    
#print words that weren't added successfully 
for line in error_words:
    print(line)